In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataloader import RadarImageTargetSet
from sklearn.model_selection import StratifiedKFold
ds = RadarImageTargetSet()

In [ ]:
boxes = []
for (img, tgt) in ds:
    for box in tgt['boxes']:
        boxes.append(box)
boxes = np.array(boxes)
box_xsizes = boxes[:, 2] - boxes[:, 0]
box_ysizes = boxes[:, 3] - boxes[:, 1]
max_x_size = int(max(box_xsizes))
max_y_size = int(max(box_ysizes))

print(max_x_size, max_y_size)

In [ ]:
# prepare training data for DopplerNet like model

# box cutouts
x = []
# box center positions in radar frame
c = []
# classes in one hot encoding
y = []
for (img, tgt) in ds:
    #print(img.shape)
    for (class_type, (y0, x0, y1, x1)) in zip(tgt['labels'], list(tgt['boxes'])):
        if np.isclose(class_type, 3.0): 
            continue
        c.append([0.5 * (x0 + x1), 0.5 * (y0 + y1)])
        # cut box out of image
        #print(class_type)
        #print(x0, y0, x1, y1)
        cutout = img[int(x0):int(x1), int(y0):int(y1)]
        #print(cutout.shape)
        if cutout.shape[0] > max_x_size and cutout.shape[1] > max_y_size:
            print("can't process box of shape", x0, y0, x1, y1, cutout.shape)
            continue
        # padding putting it in upper left corner
        # TODO: center
        y_padding = int(max_y_size - cutout.shape[0])
        x_padding = int(max_x_size - cutout.shape[1])
        padded = np.pad(cutout, ((y_padding // 2, y_padding - (y_padding // 2)), (x_padding // 2, x_padding - (x_padding // 2))), mode='constant', constant_values=0.0)
        x.append(padded)
        # class as one hot encoding
        y.append([1.0 if int(class_type) == i else 0.0 for i in range(4)])

x = np.array(x)
c = np.array(c)
y = np.array(y)
print(x.shape)
print(c.shape)
print(y.shape)

In [ ]:
plt.imshow(x[28])

In [ ]:
from sklearn.model_selection import train_test_split 
x_train,x_test,c_train,c_test,y_train,y_test=train_test_split(x, c, y, test_size=0.33, random_state=42)

x_train=np.asarray(x_train)
c_train=np.asarray(c_train)
y_train=np.asarray(y_train)
x_test=np.asarray(x_test)
c_test=np.asarray(c_test)
y_test=np.asarray(y_test)
x_train=np.reshape(x_train,[-1,11,27,1])
c_train=np.reshape(c_train,[-1,2,1])
x_test=np.reshape(x_test,[-1,11,27,1])

In [ ]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.layers import Dense, Input, Conv2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers
#from k.layers.Input as Input
#from k.layers.Conv2D as Conv2D

#model=k.Sequential()
#model.add(tf.keras.Input(shape=(11,27,1)))
#model.add(k.layers.Conv2D(32,3,3,padding='valid',
#    dilation_rate=(1, 1),
#    activation="relu"))
#model.add(k.layers.Flatten())
#model.add(k.layers.Dense(64,activation="relu"))
#model.add(k.layers.Dense(64,activation="relu"))
#model.add(k.layers.Dense(64,activation="relu"))
#model.add(k.layers.Dense(64,activation="relu"))
#model.add(k.layers.Dense(4,activation="softmax"))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()
#TODO: parallel input of absolute box position 
ridge = 5e-5
dropout = 0
# define two sets of inputs
inputA = Input(shape=(11,27,1))
inputB = Input(shape=(2,))
# the first branch operates on the first input
x = k.layers.Conv2D(32,3,3,padding='same',
    dilation_rate=(1, 1),
    activation="relu",
    kernel_regularizer=regularizers.l2(ridge),
    bias_regularizer=regularizers.l2(ridge))(inputA)
# x = Dropout(dropout)(x)
x = k.layers.Conv2D(32,3,3,padding='same',
    dilation_rate=(1, 1),
    activation="relu",
    kernel_regularizer=regularizers.l2(ridge),
    bias_regularizer=regularizers.l2(ridge))(x)
# x = Dropout(dropout)(x)
x = k.layers.Flatten()(x)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(1, activation="linear")(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(64, activation="relu",    
    kernel_regularizer=regularizers.l2(ridge),
    bias_regularizer=regularizers.l2(ridge))(combined)
z = Dense(64, activation="relu",
    kernel_regularizer=regularizers.l2(ridge),
    bias_regularizer=regularizers.l2(ridge))(z)
z = Dense(64, activation="relu",
    kernel_regularizer=regularizers.l2(ridge),
    bias_regularizer=regularizers.l2(ridge))(z)
z = Dense(4, activation="softmax")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
EPOCHS = 200
history=model.fit(x = [x_train, c_train], y = y_train, epochs=EPOCHS,batch_size=374, validation_data=([x_test, c_test], y_test)) 
# set batch size to number of images in dataset ==> slower training but minority class is consicerd in every parameter update

In [ ]:
y_pred=history.model.predict([x_test, c_test])



In [ ]:
#plt.plot(range(1,EPOCHS+1),history.history['loss'], label='validation loss')

#plt.plot(range(1,EPOCHS+1),history.history['accuracy'], label='validation accuracy')
#plt.legend()
#plt.show(
# plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.ylim((0, 10))
plt.show()

In [ ]:
history.model.evaluate([x_test, c_test] ,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_abs = np.argmax(y_pred, axis=1)
y_test_abs = np.argmax(y_test, axis=1)
cm=confusion_matrix(y_test_abs,y_pred_abs)
plt.imshow(cm)
print(cm)

print(y_pred[0])
print(y_test[0])

In [ ]:
# 10-fold cross validation
if __name__ == "__main__":
    n_folds = 10
    data, labels, header_info = load_data()
    skf = StratifiedKFold(labels, n_folds=n_folds, shuffle=True)

    for i, (train, test) in enumerate(skf):
            print("Running Fold", i+1, "/", n_folds)
            model = None # Clearing the NN.
            model = create_model()
            train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])